In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd

db_1 = list()
db_2 = list()

cols_1 = ['브랜드', '상품명', '가격', '리뷰수', '상품평점', '닉네임', '리뷰', '별점', '날짜', '맞춤정보','선택옵션', '한줄평']
cols_2=['스타일최고','구매욕상승','궁금증해소']

Zigzag_1 = pd.DataFrame(columns=cols_1)
Zigzag_1 = pd.DataFrame(columns=cols_2)

#url = 'https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id=560&sub_category_id=561&sort=201' # 미니스커트
url='https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id=507&sort=201'#원피스

driver = webdriver.Chrome()
driver.get(url)

scrolls = 3
for _ in range(scrolls):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

soup = BeautifulSoup(driver.page_source, "html.parser")
links = soup.select("#__next > div.zds-themes.light-theme > main > section.css-baq3dp.eabfyam0 > div a")

for link in links:
    product_url = link.get('href')
    full_url = f'https://zigzag.kr{product_url}'
    driver.get(full_url)
    time.sleep(5)

    item_page = BeautifulSoup(driver.page_source, 'html.parser')

    brand_name = item_page.select_one('.css-1clkw4t').text
    item_name = item_page.select_one('.css-jdi3f5').text
    price = item_page.select_one('.css-15ex2ru').text
    review_count = item_page.select_one('.css-1t4skuu').text
    item_score_avg = item_page.select_one('.e1cf25cj0').text
    
    #print(brand_name, item_name, price, review_count, item_score_avg)
    
    driver.find_element(By.CSS_SELECTOR, '#__next > div.zds-themes.light-theme > div > div.pdp__bottomFloating > div > div.css-qpt0u3.e1yh52zv2 > div > div').click()
    time.sleep(4)
    
    for _ in range(scrolls):
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(4)
    
    more_list = driver.find_elements(By.CSS_SELECTOR, ".css-1aa4nqt")
    
    for more in more_list:
        try:
            more.click()
        except:
            pass
    
    time.sleep(10)    
    
    review_page = BeautifulSoup(driver.page_source, 'html.parser')
    
    for ch in driver.find_elements(By.CSS_SELECTOR,'.css-1bpeccg'):
        filter=''.join(ch.text.split())
        
        #print(filter)

        if '스타일최고' in filter:
            style_score = int(ch.text.split('\n')[ch.text.split('\n').index('스타일최고')+1])
        else:style_score=0

        if '구매욕상승' in filter:
            purchase_score = int(ch.text.split('\n')[ch.text.split('\n').index('구매욕상승')+1])
        else:purchase_score=0

        if '궁금증해소' in filter:
            curiosity_score = int(ch.text.split('\n')[ch.text.split('\n').index('궁금증해소')+1])
        else:curiosity_score=0   

        db_2.append([style_score,purchase_score,curiosity_score])

    for review_element in review_page.select('.e1pdzmd01'):

        style_score=0
        purchase_score=0
        curiosity_score=0

        nickname = review_element.select_one('.css-utqis4').text
        review = review_element.select_one('.css-epr5m6').text
        star_container = review_element.select_one('div.css-c45d1y.e3ggphk2')
        rating = len(str(star_container).rsplit('#FFA93B')) - 1
        review_date = review_element.select_one('.css-1w6topb').text

        review_html = str(review_element.select_one('.css-q2mnm3')) 
        recommend_html = str(review_page.select('.css-1bpeccg')) 

        if '맞춤 정보' in review_html:
            custom_info=review_element.select_one('.css-q2mnm3').select_one('.css-s53yvn').text
        else:
            custom_info = None

        if '선택 옵션' in review_html:
            try:
                custom_info_element = review_element.select_one('.css-q2mnm3')
                selected_option=custom_info_element.select_one('.css-1ccr5bn').text+custom_info_element.select('.css-1ccr5bn')[1].text
            except IndexError :
                selected_option=custom_info_element.select_one('.css-1ccr5bn').text
        else:
            selected_option = None

        if '한줄평' in review_html:
            #try:
                summary_comment=review_page.select('.css-s53yvn')[2].text
            #except IndexError :
            #   summary_comment=review_page.select('.css-s53yvn')[0].text
        else:
            summary_comment = None
 

        db_1.append([brand_name, item_name, price, review_count, item_score_avg, nickname, review, rating, review_date, custom_info,selected_option,summary_comment])

    

Zigzag_1 = pd.DataFrame(db_1, columns=cols_1)
Zigzag_2 = pd.DataFrame(db_2, columns=cols_2)
Zigzag = pd.concat([Zigzag_1, Zigzag_2], axis=1)
Zigzag

Zigzag.to_csv('zigzag_data.csv', index=False)


In [137]:

Zigzag.to_excel('zigzag_data.xlsx', index=False)